# Parzen Window Classifiers

## A brief overview 

Parzen window estimation, or kernel density estimation, refers to a nonparametric technique for estimating probability density functions from sample patterns.

The Parzen or kernel estimate of the density at \( x \) is given by

$$
\hat{p}(x) = \frac{1}{n h} \sum_{i=1}^{n} K\left(\frac{x - x_i}{h}\right),
$$

where 

- $K$ is a window or a Kernel function with $K(x) > 0$ and
$\int_{-\infty}^{\infty} K(x) \, dx = 1$
- $h$ is the window width or smoothing parameter
- $n$ represents the number of data examples

In classifiers based on Parzen-window estimation, the densities are estimated for each category using the above equation, and a test point is assigned the class corresponding to the maximum posterior.

The bandwith $h$ and the Kernel function are the most important hyperparameters to these types of classifiers, with the bandwidth being the most critical choice. If $h$ is too small, the estimate would be noisy, and if it is too large, the result is oversmoothed.

Reference: ["Classifier Design with Parzen Windows" from Jain & Ramaswami, 1988](https://www.sciencedirect.com/science/article/abs/pii/B9780444871374500217#fn1).



## Application to this project

For this project, the multivariate product estimation method was adopted, described by the equation

$$
  P(x, w_k) = \frac{1}{N}\frac{1}{h^p} 
    \sum \limits _{i=1}^N 
        \displaystyle 
          \prod \limits _{j=1}^p
          K_j{
            \Bigg \{
              \frac{x_{j} - x_{ij}}{h}
            \Bigg \}
          }.
$$

This method is more robust because it does not assume the independence of the variables. Additionaly, the gaussian kernel was chosen because the entire SPECTF dataset contains only continuous variables. The equation then becomes

$$
  P(x, w_k) = \frac{1}{N}\frac{1}{h^p} 
    \sum \limits _{i=1}^N 
        \displaystyle 
          \prod \limits _{j=1}^p
          \frac{1}{ (2\pi)^{\frac{1}{2}}}
          exp{
            \Bigg \{
            -\frac{1}{2}
              \Bigg \{
                \frac{x_{j} - x_{ij}}{h}
              \Bigg \}^2
            \Bigg \} 
          },
$$

which corresponds to the final probability density function estimator for this classifier.

In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np

class ParzenClassifier(ClassifierMixin, BaseEstimator):
    """
    Parzen-based classifier using Gaussian kernel density estimation for classification problems.

    Parameters
    bandwidth (float): The smoothing parameter or window width for the Parzen window.
    n_class (int): The number of unique classes in the dataset.
    **params (dict): Additional parameters.

    Attributes
    means (np.array): List containing the mean of features for each class.
    x_class (list of np.array): List of feature arrays, one for each class, containing training examples.
    bandwidth (float): The window width for the Parzen estimator.
    """

    def __init__(self, bandwidth=1.0, n_class=2, **params):
        super().__init__()
        self.n_class = n_class
        self.means = []
        self.x_class = []
        self.bandwidth = bandwidth 

    def fit(self, X: np.array, y: np.array):
        """
        Selects the data corresponding to each class and stores it.

        Parameters
        X (np.array): The training data with shape (n_samples, n_features)
        y (np.array):  Class labels for each sample in X

        Returns
        self (object): Fitted estimator
        """
        for c in range(self.n_class):
            _X = X[y == c]
            if _X.shape[0] > 0:
                self.means.append(_X.mean(0))
                self.x_class.append(_X)
        return self

    def prod_class(self, c, h):
        """
        This method is the core of the classifier, computing the probability density estimate for each class according to the stipulated equation.

        Parameters
        c (int): Class index.
        h (float): Bandwidth for the Gaussian kernel.

        Returns
        prod (function): A function that takes data X and returns Parzen densities for class c.
        """
        def prod(X):
            n, p = self.x_class[c].shape  # Number of samples (n) and features (p)
            dif = np.abs(X - self.x_class[c]) / h  # Compute the differences scaled by h
            dif_gaussian = np.exp(-0.5 * (dif ** 2)) / ((2 * np.pi) ** (1 / 2))  # Gaussian kernel formula
            dif_parzen = np.prod(dif_gaussian, 1)  # Product over all features
            return np.sum(dif_parzen) / (n * (h ** p))  # Normalize by N and h^p
        
        return prod

    def prod_windows_parzen(self, X):
        """
        Computes the Parzen density for all classes over input data X.

        Parameters
        X (np.array): The input data.

        Returns
        parzen_estimates (np.array): Density scores for each class and each sample.
        """

        h = self.bandwidth

        parzen_estimates = np.array([np.apply_along_axis(self.prod_class(i, h), axis=1, arr=X) for i in range(self.n_class)]).T
        
        return parzen_estimates

    def predict(self, X):
        """
        Predicts the class labels for the provided data.

        Parameters
        X (np.array): Input data.

        Returns
        predicted_labels (np.array): Predicted class labels for each sample.
        """

        predicted_labels = self.prod_windows_parzen(X).argmax(1)
        
        return predicted_labels


A simple example of usage can be found below.

In [13]:
# Usage example
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

dataset = pd.read_csv('../data/preprocessed/SPECTF_preprocessed.csv')

# dataset.sample(frac=0.1) # Uncomment this if you want a subsample

# Get the features and target variable
X = dataset.drop(columns='target')
y = dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X,y)

parzen_cl = ParzenClassifier(bandwidth=0.9)

parzen_cl.fit(X_train,y_train)

y_pred = parzen_cl.predict(X_test)

print(classification_report(y_test,y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.10      0.45      0.16        11
           1       0.65      0.20      0.30        56

    accuracy                           0.24        67
   macro avg       0.37      0.33      0.23        67
weighted avg       0.56      0.24      0.28        67

[[ 5  6]
 [45 11]]
